### Restart and Run All Cells

In [1]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2022
quarter = 4

In [2]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [3]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 4) 
OR (year = 2022-1 AND quarter >= 4+1)
ORDER BY year DESC, quarter DESC



In [4]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.head()

,name,year,quarter,q_amt,Counter
0,ACE,8088,10,1281979,4
1,ADVANC,8088,10,26011284,4
3,AH,8088,10,1823648,4
4,AIE,8088,10,-22132,4
5,AIMIRT,8088,10,688432,4


In [5]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 4-1) 
OR (year = 2022-1 AND quarter >= 4) 
ORDER BY year DESC, quarter DESC


In [6]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
0,ACE,8087,10,"1,455,601",4
1,ADVANC,8087,10,"25,511,368",4
2,AEONTS,8087,10,"4,032,728",4
3,AH,8087,10,"1,541,418",4
4,AIE,8087,10,"176,807",4


In [7]:
dfp.name.unique().shape

(233,)

In [8]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlt)
stocks.shape

(233, 15)

In [9]:
sql = """
SELECT *
FROM tickers
"""
tickers = pd.read_sql(sql, conlt)
tickers.shape

(401, 9)

In [10]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,ACE,2022,Q4,"1,281,979","1,455,601","-173,622",-11.93%
1,ADVANC,2022,Q4,"26,011,284","25,511,368","499,916",1.96%
2,AH,2022,Q4,"1,823,648","1,541,418","282,230",18.31%
3,AIE,2022,Q4,"-22,132","176,807","-198,939",-112.52%
4,AIMIRT,2022,Q4,"688,432","693,015","-4,583",-0.66%


In [11]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'
"""
sql = sql % (year, quarter)
print(sql)

rp = conlt.execute(sql)
rp.rowcount


DELETE FROM qt_profits 
WHERE year = 2022 AND quarter = 'Q4'



208

In [12]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.head()

,name,id
0,2S,727
1,A,1
2,AAV,732
3,ACE,698
4,ADVANC,6


In [13]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
df_ins.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
0,ACE,2022,Q4,"1,281,979","1,455,601","-173,622",-11.93%,698
1,ADVANC,2022,Q4,"26,011,284","25,511,368","499,916",1.96%,6
2,AH,2022,Q4,"1,823,648","1,541,418","282,230",18.31%,9
3,AIE,2022,Q4,"-22,132","176,807","-198,939",-112.52%,720
4,AIMIRT,2022,Q4,"688,432","693,015","-4,583",-0.66%,669


In [14]:
rcds = df_ins.values.tolist()
len(rcds)

210

In [15]:
#for rcd in rcds:
#    print(rcd)

In [16]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, \
inc_amt, inc_pct, ticker_id) \
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [17]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [18]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,ACE,2022,Q4,"1,281,979","1,455,601","-173,622",-11.93%
1,ADVANC,2022,Q4,"26,011,284","25,511,368","499,916",1.96%
2,AH,2022,Q4,"1,823,648","1,541,418","282,230",18.31%
4,AIMIRT,2022,Q4,"688,432","693,015","-4,583",-0.66%
5,AIT,2022,Q4,"541,645","582,222","-40,577",-6.97%


In [19]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,ACE,2022,Q4,"1,281,979","1,455,601","-173,622",-11.93%
1,ADVANC,2022,Q4,"26,011,284","25,511,368","499,916",1.96%
2,AH,2022,Q4,"1,823,648","1,541,418","282,230",18.31%
4,AIMIRT,2022,Q4,"688,432","693,015","-4,583",-0.66%
5,AIT,2022,Q4,"541,645","582,222","-40,577",-6.97%


In [20]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
2,AH,2022,Q4,"1,823,648","1,541,418","282,230",18.31%
8,ANAN,2022,Q4,"-295,785","-560,806","265,021",47.26%
9,AOT,2022,Q4,"-11,087,867","-14,912,794","3,824,927",25.65%
14,ASW,2022,Q4,"1,118,522","905,705","212,817",23.50%
15,AWC,2022,Q4,"3,853,861","3,414,980","438,881",12.85%


In [21]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
2,AH,2022,Q4,"1,823,648","1,541,418","282,230",18.31%
14,ASW,2022,Q4,"1,118,522","905,705","212,817",23.50%
15,AWC,2022,Q4,"3,853,861","3,414,980","438,881",12.85%
31,BH,2022,Q4,"4,938,222","4,004,426","933,796",23.32%
45,CPN,2022,Q4,"10,759,886","9,769,856","990,030",10.13%


In [22]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
110,MINT,2022,Q4,"4,286,373","818,393","3,467,980",423.75%,301
73,HANA,2022,Q4,"2,102,071","970,153","1,131,918",116.67%,205
99,LHFG,2022,Q4,"1,578,755","860,959","717,796",83.37%,268
78,III,2022,Q4,"785,358","492,038","293,320",59.61%,637
153,SIRI,2022,Q4,"4,279,885","2,831,419","1,448,466",51.16%,447


In [23]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
2,AH,2022,Q4,"1,823,648","1,541,418","282,230",18.31%,9
14,ASW,2022,Q4,"1,118,522","905,705","212,817",23.50%,728
15,AWC,2022,Q4,"3,853,861","3,414,980","438,881",12.85%,699
31,BH,2022,Q4,"4,938,222","4,004,426","933,796",23.32%,61
45,CPN,2022,Q4,"10,759,886","9,769,856","990,030",10.13%,121


In [24]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
2,AH,2022,Q4,"1,823,648","1,541,418","282,230",18.31%,9
14,ASW,2022,Q4,"1,118,522","905,705","212,817",23.50%,728
15,AWC,2022,Q4,"3,853,861","3,414,980","438,881",12.85%,699
31,BH,2022,Q4,"4,938,222","4,004,426","933,796",23.32%,61
45,CPN,2022,Q4,"10,759,886","9,769,856","990,030",10.13%,121


In [25]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name



,name,Nbr. of Group of 4 Quarters
